This notebook generates the following panels:
- ( ) Example of 1 neuron - Closed loop: example raster, PSTH, depth tuning

use env 2p_analysis_cottage

In [ ]:
VERSION = 1


In [ ]:
%reload_ext autoreload
%autoreload 2


from functools import partial

print = partial(print, flush=True)

import functools
print = functools.partial(print, flush=True)

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import pickle
from tqdm import tqdm
import scipy

import flexiznam as flz
from cottage_analysis.preprocessing import synchronisation
from cottage_analysis.analysis import spheres, find_depth_neurons, pipeline_utils, common_utils, fit_gaussian_blob, size_control
from cottage_analysis.plotting import basic_vis_plots

In [ ]:
# example neurons:
# 'PZAH8.2f_S20230109': [409, 571,238,724]
# 'PZAH8.2h_S20230113': [174, 261, 249, 54, 86, 88]

In [ ]:
project='hey2_3d-vision_foodres_20220101'
session_name='PZAH8.2h_S20230116'
flexilims_session = flz.get_flexilims_session(project)

vs_df_all, trials_df_all = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={'anatomical_only':3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)

neurons_ds = pipeline_utils.create_neurons_ds(
    session_name=session_name, flexilims_session=flexilims_session, project=None, conflicts="skip"
)
neurons_df = pd.read_pickle(neurons_ds.path_full)



In [ ]:
# RASTER PLOT OF AN EXAMPLE NEURON
plt.figure(figsize=(2*8,5))
basic_vis_plots.plot_raster_all_depths(
    neurons_df=neurons_df,
    trials_df=trials_df_all,
    roi=54,
    is_closed_loop=True,
    max_distance=6,
    nbins=100,
    frame_rate=15,
    vmax=1,
    plot=True,
    fontsize_dict={'title':15, 'label':10, 'tick':10},
);

In [ ]:
# DEPTH TUNING & PSTH OF 4 EXAMPLE NEURONS
plt.figure(figsize=(5*4,5*2))
fontsize_dict={'title':15, 'label':10, 'tick':10}
rois = [249, 54, 250, 88]
for i, roi in enumerate(rois):
    plt.subplot2grid((2,4),(0,i))
    basic_vis_plots.plot_depth_tuning_curve(
        neurons_df=neurons_df,
        trials_df=trials_df_all,
        roi=roi,
        rs_thr=0.2,
        plot_fit=False,
        linewidth=3,
        linecolor="k",
        fit_linecolor="r",
        closed_loop=1,
        fontsize_dict=fontsize_dict
    )
    

    plt.subplot2grid((2,4),(1,i))
    basic_vis_plots.plot_PSTH(
        neurons_df=neurons_df,
        trials_df=trials_df_all,
        roi=roi,
        is_closed_loop=True,
        max_distance=6,
        nbins=15,
        frame_rate=15,
        linewidth=3,
        fontsize_dict=fontsize_dict
    )

In [ ]:
# DEPTH TUNING & SPEED TUNINGS & RSOF matrices OF 3 EXAMPLE NEURONS
plt.figure(figsize=(5*4,5*3))
fontsize_dict={'title':15, 'label':10, 'tick':10}
rois = [249, 250, 88]
for i, roi in enumerate(rois):
    plt.subplot2grid((3,4),(i,0))
    basic_vis_plots.plot_depth_tuning_curve(
        neurons_df=neurons_df,
        trials_df=trials_df_all,
        roi=roi,
        rs_thr=0.2,
        plot_fit=False,
        linewidth=3,
        linecolor="k",
        fit_linecolor="r",
        closed_loop=1,
        fontsize_dict=fontsize_dict
    )
    
    plt.subplot2grid((3,4),(i,1))
    basic_vis_plots.plot_speed_tuning(
        neurons_df=neurons_df,
        trials_df=trials_df_all,
        roi=roi,
        is_closed_loop=1,
        nbins=10,
        which_speed="RS",
        speed_min=0.01,
        speed_max=1.5,
        speed_thr=0.01,
        smoothing_sd = 1
    )
    
    plt.subplot2grid((3,4),(i,2))
    basic_vis_plots.plot_speed_tuning(
        neurons_df=neurons_df,
        trials_df=trials_df_all,
        roi=roi,
        is_closed_loop=1,
        nbins=10,
        which_speed="OF",
        speed_min=0.01,
        speed_max=1.5,
        speed_thr=0.01,
        smoothing_sd = 1
    )
    
    plt.subplot2grid((3,4),(i,3))
    basic_vis_plots.plot_RS_OF_matrix(trials_df=trials_df_all, 
                                      roi=roi, 
                                      log_range=log_range, 
                                      fontsize_dict=fontsize_dict);